In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Avvio della sessione Spark
spark = SparkSession.builder.appName("WeightedSymptomClassification").getOrCreate()


In [ ]:
# Caricamento dataset con gravità dei sintomi
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/drive/MyDrive/BigData/dataset_weighted.csv")

# Colonne sintomi
symptom_cols = [col_name for col_name in df.columns if "Symptom_" in col_name]


In [ ]:
# Indicizzazione della colonna Disease
indexer = StringIndexer(inputCol="Disease", outputCol="label")
df = indexer.fit(df).transform(df)


In [ ]:
# Assemblaggio delle feature sintomatiche
assembler = VectorAssembler(inputCols=symptom_cols, outputCol="features")
df = assembler.transform(df)


In [ ]:
# Suddivisione del dataset
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)


In [ ]:
# Random Forest con sintomi pesati
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100)
model = rf.fit(train_data)
predictions = model.transform(test_data)


In [ ]:
# Accuracy su test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy con sintomi pesati:", accuracy)


Accuracy: ~93%

è stato ottenuto un miglioramento enorme passando:
*	da 19% (feature binarie 0/1),
*	a 93% usando i pesi di gravità dei sintomi.

# **Cosa significa?**
* I dati, non il modello, erano il problema.
*	Aggiungendo feature più informative (severity), il modello è stato in grado di distinguere molto meglio le malattie.
*	Questo è l’esempio perfetto di Data-Centric AI: il boost non è arrivato da un nuovo algoritmo, ma da un dataset curato meglio.